In [21]:
MYHFKEY = "..."

# Initialize Notebook

In [2]:
!pip install langchain
!pip install huggingface_hub
!pip install transformers
!pip install fairseq

In [3]:
import os
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import warnings
import requests

warnings.filterwarnings('ignore')

In [4]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = MYHFKEY
headers = {"Authorization": "Bearer " + MYHFKEY}

# BART uses for summarising a text

## Texts to summarize

In [5]:
frText = "Un pingouin décide de partir en vacances. Il fait beau, le soleil brille, et il pense que c'est le moment idéal pour prendre un peu de bon temps. \
Il réserve un billet pour Hawaï et s'envole joyeusement vers cette destination ensoleillée. Arrivé à Hawaï, le pingouin se rend sur la plage et \
s'installe confortablement sur sa serviette. Il décide de profiter du soleil et de faire une petite sieste. Alors qu'il est en train de se détendre, \
il sent quelque chose de mouillé sur son bec. Il ouvre les yeux et découvre qu'un toucan est en train de lui faire de l'ombre avec son énorme bec coloré. \
Étonné, le pingouin se redresse et dit : Eh bien, qu'est-ce que tu fais ici ? Les toucans ne vivent pas à Hawaï ! \
Le toucan répond avec un sourire : Eh bien, mon ami, moi aussi, je suis en vacances ! \
Le pingouin rit et accepte la compagnie du toucan. Les deux amis passent la journée à siroter des cocktails, à danser sur la plage, et à profiter de la belle île. \
Le lendemain, le pingouin décide de rentrer chez lui. Il remercie le toucan pour la belle journée et prend un vol de retour vers sa banquise. \
Quand il arrive chez lui, ses amis pingouins le regardent avec étonnement. Où étais-tu ? demandent-ils. \
Le pingouin sourit et répond : J'étais à Hawaï, et j'ai même rencontré un toucan en vacances là-bas ! \
Ses amis le regardent incrédules et lui demandent : Un toucan à Hawaï ? C'est une blague, non ? \
Le pingouin rit et dit : Eh bien, parfois, il faut un peu de fantaisie pour rendre la vie plus amusante ! \
Et c'est ainsi que le pingouin raconta à ses amis la plus étrange, mais aussi la plus drôle, des aventures de vacances à Hawaï avec un toucan exotique."

In [6]:
enText = "I was driving on the the long district road in Las vegas close to the Bellagio hotel. I was driven at 50 miles per hour when suddenly a man decided \
to cross the road without notice. I would like to precise that man jumped and ran directly to my car, this man was just crazy or maybe drunk, i don't know. \
It was approximately 2:30PM when the accident occured. I immedialtely stopped the car to see how was the man after the collision. \
Hopefully he was able to stand and told me he was fine."

Text size:

In [7]:
len(frText)

1657

## (Method 1,2 & 3) Using Hugging Face

### (Method 1) Via Python calls

We'll use the facebook/bart-large-cnn model which was designed for summary purpose.

See more details here: https://huggingface.co/facebook/bart-large-cnn

In [8]:
template = """ {text}"""
prompt_template = PromptTemplate(input_variables=["text"], template=template)
llm=HuggingFaceHub(repo_id="facebook/bart-large-cnn")
chain = LLMChain(llm=llm, prompt=prompt_template)
exceprt = chain.run(frText)
print(exceprt)

Un pingouin réserve un billet pour Hawaï et s'envole joyeusement vers cette destination ensoleillée. Il ouvre les yeux et découvre qu'un toucan est en train de lui faire de l'ombre avec son énorme bec coloré.


In [9]:
len(exceprt)

208

### (Method 2) Via API calls

In [10]:
import requests

API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output = query({
	"inputs": frText,
})

In [11]:
output

[{'summary_text': "Un pingouin réserve un billet pour Hawaï et s'envole joyeusement vers cette destination ensoleillée. Il ouvre les yeux et découvre qu'un toucan est en train de lui faire de l'ombre avec son énorme bec coloré. Les deux amis passent la journée à siroter des cocktails."}]

In [12]:
len(output[0]['summary_text'])

266

### (Method 3) Then use pipeline API

In [13]:
from transformers import pipeline
summarizer = pipeline("summarization",
                      model="facebook/bart-large-cnn")

In [14]:
response = summarizer(frText, max_length=200, min_length=30, do_sample=False)
response

[{'summary_text': "Un pingouin réserve un billet pour Hawaï et s'envole joyeusement vers cette destination ensoleillée. Il ouvre les yeux et découvre qu'un toucan est en train de lui faire de l'ombre avec son énorme bec coloré. Les deux amis passent la journée à siroter des cocktails."}]

In [15]:
len(response[0]['summary_text'])

266

## (Method 4) Now use pyTorch and the META published libs

This method is interresting because it can run locally as we'll download the entire model (this is usually the first step before fine-tuning ;-))

See here for more details : https://github.com/facebookresearch/fairseq/tree/main/examples/bart

Download first the BART model (approx 3,5 Gb) !

In [16]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/bart.large.tar.gz
!tar -xzvf bart.large.tar.gz

--2023-11-10 09:33:08--  https://dl.fbaipublicfiles.com/fairseq/models/bart.large.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.11, 3.162.163.19, 3.162.163.34, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3699866548 (3.4G) [application/x-tar]
Saving to: ‘bart.large.tar.gz.1’

bart.large.tar.gz.1   2%[                    ]  90.06M  22.0MB/s    eta 2m 24s ^C
bart.large/
bart.large/model.pt
^C


Now use the Meta fairseq library (https://github.com/facebookresearch/fairseq)

fairseq "is a sequence modeling toolkit that allows researchers and developers to train custom models for translation, summarization, language modeling and other text generation tasks."

In [ ]:
!pip install fairseq

In [18]:
from fairseq.models.bart import BARTModel
bart = BARTModel.from_pretrained('bart.large', checkpoint_file='model.pt')
hypotheses_batch = bart.sample([frText],
                               beam=4,
                               lenpen=2.0,
                               max_len_b=140,
                               min_len=55,
                               no_repeat_ngram_size=3)
print(hypotheses_batch)

1042301B [00:00, 3984083.86B/s]
456318B [00:00, 9743208.61B/s]


["Un pingouin décide de partir en vacances. Il réserve un billet pour Hawaï et s'envole joyeusement vers cette destination ensoleillée. Arrivé à Hawaï, le pingouin se rend sur la plage et s'installe confortablement sur sa serviette. Il fait beau, le soleil brille, et il pense que c'est le moment idéal pour prendre un peu de bon temps. Il décide de profiter du soleil et de faire une petite sieste. Le lendemain, le"]


In [20]:
len(hypotheses_batch[0])

414